In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df1_WS3T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment1_WS3_has_duration.csv")

In [3]:
df1_WS3T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1079339|441319|682219,Assessment Outcome - Successful,Assessment Outcome - Successful,11/5/2019 14:06:25,ATKINSONM,Faculty,18,N,2. Perform Assessment,Part 3 - Assessment Outcome,...,MCW,HDCW,Full Degree,OF,OF,Y,$DEF,Y,0 days 0 hours 0 minutes 7 seconds,7.0


In [4]:
numOfWorkflowProcesses = df1_WS3T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS3T[df1_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS3T[df1_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS3T[df1_WS3T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED']=='Release Offer']
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS3T[df1_WS3T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [11]:
s1_t3=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s1_t3.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step
0,1,1531,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Assessment Outcome - Not Successful}
1,2,462,14 days 7 hours 57 minutes 0 seconds,0.0 days 0.0 hours 1.0 minutes 55.5 seconds,0.32%,"{OC - Reviewed - Other, Assessment Decision - ..."
2,3,4500,1 days 16 hours 11 minutes 45 seconds,0.0 days 0.0 hours 2.0 minutes 15.0 seconds,30.33%,"{OC - Reviewed - Other, AR - Satisfied - Other..."
3,4,2347,2 days 17 hours 16 minutes 35 seconds,0.0 days 0.0 hours 2.0 minutes 43.0 seconds,22.48%,"{OC - Reviewed - Other, AR - Reviewed - furthe..."
4,5,291,27 days 15 hours 2 minutes 7 seconds,0.0 days 0.0 hours 8.0 minutes 14.0 seconds,9.42%,"{OC - Reviewed - Other, AR - Requested - sylla..."
5,6,111,38 days 16 hours 7 minutes 49 seconds,2.0 days 3.0 hours 49.0 minutes 6.0 seconds,6.31%,"{OC - Reviewed - Other, AR - Satisfied - appli..."
6,7,50,52 days 14 hours 49 minutes 22 seconds,19.0 days 5.0 hours 0.0 minutes 48.5 seconds,4.0%,"{OC - Reviewed - Other, AR - Satisfied - appli..."
7,8,32,70 days 0 hours 7 minutes 34 seconds,39.0 days 9.0 hours 19.0 minutes 13.0 seconds,4.3%,"{Assessment Decision - No Decision, PREPARE TO..."
8,9,16,83 days 18 hours 55 minutes 54 seconds,71.0 days 11.0 hours 33.0 minutes 28.5 seconds,2.78%,"{OC - Reviewed - Other, AR - Satisfied - Other..."
9,10,9,76 days 7 hours 6 minutes 33 seconds,5.0 days 0.0 hours 57.0 minutes 6.0 seconds,3.33%,"{Assessment Decision - No Decision, OC - Creat..."


In [12]:
s1_t3.to_csv('segment1_WS3_table.csv', index=False)









